# Segmenting and Clustering Neighborhoods in Toronto

In [ ]:
import pandas as pd
import requests
#!conda install -c conda-forge lxml --yes
import lxml

# Part 1: Create the DataFrame

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
r = requests.get(url)

## 1. Scraping Toronto data with the help of Pandas library

In [4]:
dataframes=pd.read_html(r.text, header=0)

## 2. Get the DataFrame with the Toronto neighborhood data

In [5]:
toronto = dataframes[0]

In [6]:
toronto.head()

Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

## 3. Drop all rows with not assigned boroughs & Not assigned neighborhoods will renamed after their corresponding Borough

In [7]:
if len(toronto.loc[toronto.Borough == 'Not assigned']) == 0:
    print('None neighboorhood listed as non assigned after filtering')
else:
    print('Let\'s Drop {} non assigned Boroughs in Toronto'.format(len(toronto.loc[toronto.Borough == 'Not assigned'])))
    toronto.drop(toronto.loc[toronto.Borough == 'Not assigned'].index, inplace = True)
    toronto.reset_index(inplace = True, drop = True)
    

Let's Drop 77 non assigned Boroughs in Toronto


In [8]:
if len(toronto.loc[toronto.Neighbourhood == 'Not assigned']) == 0:
    print('None neighborhood listed as non assigned after filtering')
    



None neighborhood listed as non assigned after filtering


In [9]:
toronto.head()

Postal Code           Borough                                Neighbourhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [10]:
toronto.shape

(103, 3)

## 4. Combine Neighborhoods with the same Postal Code

### This is already fixed on the DataFrame 

## 5. Print  the shape of the DataFrame

In [11]:
toronto.shape

(103, 3)

# Part 2: Toronto Neighborhoods Location 

In [12]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
from geopy.geocoders import Nominatim 


In [13]:
#!wget 'https://cocl.us/Geospatial_data' -O 'toronto_coordinates.csv' 
print('Done')

Done


In [14]:
coord = pd.read_csv('toronto_coordinates.csv')

## Merge toronto data with the coordinates by doing an inner join on the dataframes

In [15]:
toronto_df = toronto.merge(coord, how = 'inner', left_on = 'Postal Code', right_on = 'Postal Code')

In [16]:
toronto_df

Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4          Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98       The Kingsway, Montgomery Road, Old Mill North  43.653654 -79.506944  
99                                Church and Wellesley  43.665860 -79.383160  
100  Business reply mail Processing Centre, South C...  43.662744 -79.321558  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  43.636258 -79.498509  
102  Mimico NW, The Queensway West, South of Bloor,...  43.628841 -79.520999  

[103 rows x 5 columns]

# Part 3: Explore and cluster the neighborhoods in Toronto

In [17]:
## Explore Toronto's Neighborhoods

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = 'Toronto')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
# create a Map of Toronto with the surrounding neighborhoods

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#E52B50',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Foursquare credentials

In [19]:
CLIENT_ID = '0WC5F0U5FBMNZZTRU2JNK2DPMXAMCZIALHOKO5D3GLPBW3Q3' # your Foursquare ID
CLIENT_SECRET = 'TPBRN5MV3PU5M3TFKE40KGHFBSDLZB5ZABQZLVLABPX3MRQ1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0WC5F0U5FBMNZZTRU2JNK2DPMXAMCZIALHOKO5D3GLPBW3Q3
CLIENT_SECRET:TPBRN5MV3PU5M3TFKE40KGHFBSDLZB5ZABQZLVLABPX3MRQ1


In [20]:
### Pick all boroughs that contain the word Toronto

tor_data = toronto_df.loc[toronto_df['Borough'].str.contains('Toronto')].reset_index(drop = True)

In [21]:
tor_data.head()

Postal Code           Borough                                Neighbourhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031

In [22]:
tor_data.Borough.unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

4 Boroughs were selected by this filtering

In [23]:
radius = 500
limit = 100

In [73]:
def getNearbyVenues(names, neigh, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, n, lat, lng in zip(names, neigh, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            n,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Neighbourhood',
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
toronto_venues = getNearbyVenues(names=tor_data['Borough'],
                                   neigh = tor_data['Neighbourhood'],
                                   latitudes=tor_data['Latitude'],
                                   longitudes=tor_data['Longitude']
                                  )

Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


In [76]:
toronto_venues.shape

(1644, 8)

In [77]:
toronto_venues.head()

Borough              Neighbourhood  Latitude  Longitude  \
0  Downtown Toronto  Regent Park, Harbourfront  43.65426 -79.360636   
1  Downtown Toronto  Regent Park, Harbourfront  43.65426 -79.360636   
2  Downtown Toronto  Regent Park, Harbourfront  43.65426 -79.360636   
3  Downtown Toronto  Regent Park, Harbourfront  43.65426 -79.360636   
4  Downtown Toronto  Regent Park, Harbourfront  43.65426 -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3     Body Blitz Spa East       43.654735       -79.359874   
4          Impact Kitchen       43.656369       -79.356980   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3                  Spa  
4           Restaurant

In [79]:
#Venues per Borough
toronto_venues.groupby('Neighbourhood').count().Venue

Neighbourhood
Berczy Park                                                                                                    59
Brockton, Parkdale Village, Exhibition Place                                                                   23
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                           19
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     17
Central Bay Street                                                                                             65
Christie                                                                                                       17
Church and Wellesley                                                                                           76
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                

In [80]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


### Let's create one hot encodings for the venues

In [86]:

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


toronto_onehot['Borough'] = toronto_venues['Borough'] 
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 


fixed_columns = [toronto_onehot.columns[-2]] + [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Borough              Neighbourhood  Afghan Restaurant  Airport  \
0  Downtown Toronto  Regent Park, Harbourfront                  0        0   
1  Downtown Toronto  Regent Park, Harbourfront                  0        0   
2  Downtown Toronto  Regent Park, Harbourfront                  0        0   
3  Downtown Toronto  Regent Park, Harbourfront                  0        0   
4  Downtown Toronto  Regent Park, Harbourfront                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  ...  Trail  Train Station  \
0                 0                    0  ...      0              0   
1                 0                    0  ...      0              0   
2                 0                    0  ...      0              0   
3                 0                    0  ...      0              0   
4                 0                    0  ...      0              0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                              0                 0                      0   
1                              0                 0                      0   
2                              0                 0                      0   
3                              0                 0                      0   
4                              0                 0                      0   

   Wine Bar  Wine Shop  Women's Store  Yoga Studio           Borough  
0         0          0              0            0  Downtown Toronto  
1         0          0              0            0  Downtown Toronto  
2         0          0              0            0  Downtown Toronto  
3         0          0              0            0  Downtown Toronto  
4         0          0              0            0  Downtown Toronto  

[5 rows x 236 columns]

In [87]:
toronto_freq = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_freq

Neighbourhood  Afghan Restaurant  \
0                                         Berczy Park           0.000000   
1        Brockton, Parkdale Village, Exhibition Place           0.000000   
2   Business reply mail Processing Centre, South C...           0.000000   
3   CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
4                                  Central Bay Street           0.000000   
5                                            Christie           0.000000   
6                                Church and Wellesley           0.013158   
7                      Commerce Court, Victoria Hotel           0.000000   
8                                          Davisville           0.000000   
9                                    Davisville North           0.000000   
10                       Dufferin, Dovercourt Village           0.000000   
11             First Canadian Place, Underground city           0.000000   
12    Forest Hill North & West, Forest Hill Road Park           0.000000   
13                           Garden District, Ryerson           0.000000   
14  Harbourfront East, Union Station, Toronto Islands           0.000000   
15                      High Park, The Junction South           0.000000   
16                     India Bazaar, The Beaches West           0.000000   
17          Kensington Market, Chinatown, Grange Park           0.000000   
18                                      Lawrence Park           0.000000   
19                           Little Portugal, Trinity           0.000000   
20                        Moore Park, Summerhill East           0.000000   
21                  North Toronto West, Lawrence Park           0.000000   
22                             Parkdale, Roncesvalles           0.000000   
23        Queen's Park, Ontario Provincial Government           0.000000   
24                          Regent Park, Harbourfront           0.000000   
25                           Richmond, Adelaide, King           0.000000   
26                                           Rosedale           0.000000   
27                                           Roselawn           0.000000   
28                                 Runnymede, Swansea           0.000000   
29                                     St. James Town           0.000000   
30                        St. James Town, Cabbagetown           0.000000   
31                                     Stn A PO Boxes           0.000000   
32                                    Studio District           0.000000   
33  Summerhill West, Rathnelly, South Hill, Forest...           0.000000   
34                The Annex, North Midtown, Yorkville           0.000000   
35                                        The Beaches           0.000000   
36                       The Danforth West, Riverdale           0.000000   
37           Toronto Dominion Centre, Design Exchange           0.000000   
38                     University of Toronto, Harbord           0.000000   

     Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0   0.000000            0.000000      0.000000        0.000000   
1   0.000000            0.000000      0.000000        0.000000   
2   0.000000            0.000000      0.000000        0.000000   
3   0.058824            0.058824      0.058824        0.117647   
4   0.000000            0.000000      0.000000        0.000000   
5   0.000000            0.000000      0.000000        0.000000   
6   0.000000            0.000000      0.000000        0.000000   
7   0.000000            0.000000      0.000000        0.000000   
8   0.000000            0.000000      0.000000        0.000000   
9   0.000000            0.000000      0.000000        0.000000   
10  0.000000            0.000000      0.000000        0.000000   
11  0.000000            0.000000      0.000000        0.000000   
12  0.000000            0.000000      0.000000        0.000000   
13  0.000000            0.000000      0.000000        0.000000   
14  0.000

### Find the top 5 venues per borough

In [88]:
tor_transposed = toronto_freq.iloc[:, 1:].T

In [92]:
tor_transposed.shape

(233, 39)

In [93]:
toronto.Neighbourhood.shape

(103,)

In [94]:
for i, bor in enumerate(toronto_freq.Neighbourhood, 0):
    print('For neighborhood {} the 5 most common venues and their frequencies are:'.format(bor))
    print('------------')
    print(tor_transposed[i].nlargest(5))
    print('------------')
    
    

For neighborhood Berczy Park the 5 most common venues and their frequencies are:
------------
Coffee Shop    0.084746
Bakery         0.033898
Beer Bar       0.033898
Café           0.033898
Cheese Shop    0.033898
Name: 0, dtype: float64
------------
For neighborhood Brockton, Parkdale Village, Exhibition Place the 5 most common venues and their frequencies are:
------------
Café              0.130435
Breakfast Spot    0.086957
Coffee Shop       0.086957
Bakery            0.043478
Bar               0.043478
Name: 1, dtype: float64
------------
For neighborhood Business reply mail Processing Centre, South Central Letter Processing Plant Toronto the 5 most common venues and their frequencies are:
------------
Light Rail Station    0.105263
Auto Workshop         0.052632
Brewery               0.052632
Burrito Place         0.052632
Butcher               0.052632
Name: 2, dtype: float64
------------
For neighborhood CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Qua

In [108]:
def common_venues(data, num_venues):
    cols = ['Neighbourhood']
    
    for i in range(num_venues):
        cols.append('CommonVenue' + str(i + 1))
        
    df = pd.DataFrame(columns = cols)
    row = []
    
    mydict = {}
    for i, bor in enumerate(toronto_freq.Neighbourhood, 0):
        row.append(bor)
        row.extend(data[i].nlargest(num_venues).index.to_list())
        
        for col, r in zip(cols, row):
            mydict[col] = r
        
        df = df.append(mydict, ignore_index = True)
        row = [] 
    
    
    return df
    

In [120]:
commondf = common_venues(tor_transposed, 15)

In [121]:
commondf

Neighbourhood        CommonVenue1  \
0                                         Berczy Park         Coffee Shop   
1        Brockton, Parkdale Village, Exhibition Place                Café   
2   Business reply mail Processing Centre, South C...  Light Rail Station   
3   CN Tower, King and Spadina, Railway Lands, Har...      Airport Lounge   
4                                  Central Bay Street         Coffee Shop   
5                                            Christie       Grocery Store   
6                                Church and Wellesley         Coffee Shop   
7                      Commerce Court, Victoria Hotel         Coffee Shop   
8                                          Davisville        Dessert Shop   
9                                    Davisville North      Breakfast Spot   
10                       Dufferin, Dovercourt Village              Bakery   
11             First Canadian Place, Underground city         Coffee Shop   
12    Forest Hill North & West, Forest Hill Road Park            Bus Line   
13                           Garden District, Ryerson      Clothing Store   
14  Harbourfront East, Union Station, Toronto Islands         Coffee Shop   
15                      High Park, The Junction South                Café   
16                     India Bazaar, The Beaches West                Park   
17          Kensington Market, Chinatown, Grange Park                Café   
18                                      Lawrence Park                Park   
19                           Little Portugal, Trinity                 Bar   
20                        Moore Park, Summerhill East               Trail   
21                  North Toronto West, Lawrence Park      Clothing Store   
22                             Parkdale, Roncesvalles      Breakfast Spot   
23        Queen's Park, Ontario Provincial Government         Coffee Shop   
24                          Regent Park, Harbourfront         Coffee Shop   
25                           Richmond, Adelaide, King         Coffee Shop   
26                                           Rosedale                Park   
27                                           Roselawn              Garden   
28                                 Runnymede, Swansea                Café   
29                                     St. James Town                Café   
30                        St. James Town, Cabbagetown              Bakery   
31                                     Stn A PO Boxes         Coffee Shop   
32                                    Studio District                Café   
33  Summerhill West, Rathnelly, South Hill, Forest...         Coffee Shop   
34                The Annex, North Midtown, Yorkville                Café   
35                                        The Beaches         Coffee Shop   
36                       The Danforth West, Riverdale    Greek Restaurant   
37           Toronto Dominion Centre, Design Exchange         Coffee Shop   
38                     University of Toronto, Harbord                Café   

           CommonVenue2                   CommonVenue3  \
0                Bakery                       Beer Bar   
1        Breakfast Spot                    Coffee Shop   
2         Auto Workshop                        Brewery   
3       Airport Service               Airport Terminal   
4    Italian Restaurant                           Café   
5                  Café                           Park   
6   Japanese Restaurant               Sushi Restaurant   
7                  Café                     Restaurant   
8           Pizza Place                 Sandwich Place   
9      Department Store              Food & Drink Shop   
10             Pharmacy                           Bank   
11                 Café                          Hotel   
12        Jewelry Store                           Park   
13          Coffee Shop                Bubble Tea Shop   
14             Aquarium                           Café   
15   Mexican Restaurant        

# Perform Clustering

In [122]:
from sklearn.cluster import KMeans

In [123]:
toronto_freq

Neighbourhood  Afghan Restaurant  \
0                                         Berczy Park           0.000000   
1        Brockton, Parkdale Village, Exhibition Place           0.000000   
2   Business reply mail Processing Centre, South C...           0.000000   
3   CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
4                                  Central Bay Street           0.000000   
5                                            Christie           0.000000   
6                                Church and Wellesley           0.013158   
7                      Commerce Court, Victoria Hotel           0.000000   
8                                          Davisville           0.000000   
9                                    Davisville North           0.000000   
10                       Dufferin, Dovercourt Village           0.000000   
11             First Canadian Place, Underground city           0.000000   
12    Forest Hill North & West, Forest Hill Road Park           0.000000   
13                           Garden District, Ryerson           0.000000   
14  Harbourfront East, Union Station, Toronto Islands           0.000000   
15                      High Park, The Junction South           0.000000   
16                     India Bazaar, The Beaches West           0.000000   
17          Kensington Market, Chinatown, Grange Park           0.000000   
18                                      Lawrence Park           0.000000   
19                           Little Portugal, Trinity           0.000000   
20                        Moore Park, Summerhill East           0.000000   
21                  North Toronto West, Lawrence Park           0.000000   
22                             Parkdale, Roncesvalles           0.000000   
23        Queen's Park, Ontario Provincial Government           0.000000   
24                          Regent Park, Harbourfront           0.000000   
25                           Richmond, Adelaide, King           0.000000   
26                                           Rosedale           0.000000   
27                                           Roselawn           0.000000   
28                                 Runnymede, Swansea           0.000000   
29                                     St. James Town           0.000000   
30                        St. James Town, Cabbagetown           0.000000   
31                                     Stn A PO Boxes           0.000000   
32                                    Studio District           0.000000   
33  Summerhill West, Rathnelly, South Hill, Forest...           0.000000   
34                The Annex, North Midtown, Yorkville           0.000000   
35                                        The Beaches           0.000000   
36                       The Danforth West, Riverdale           0.000000   
37           Toronto Dominion Centre, Design Exchange           0.000000   
38                     University of Toronto, Harbord           0.000000   

     Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0   0.000000            0.000000      0.000000        0.000000   
1   0.000000            0.000000      0.000000        0.000000   
2   0.000000            0.000000      0.000000        0.000000   
3   0.058824            0.058824      0.058824        0.117647   
4   0.000000            0.000000      0.000000        0.000000   
5   0.000000            0.000000      0.000000        0.000000   
6   0.000000            0.000000      0.000000        0.000000   
7   0.000000            0.000000      0.000000        0.000000   
8   0.000000            0.000000      0.000000        0.000000   
9   0.000000            0.000000      0.000000        0.000000   
10  0.000000            0.000000      0.000000        0.000000   
11  0.000000            0.000000      0.000000        0.000000   
12  0.000000            0.000000      0.000000        0.000000   
13  0.000000            0.000000      0.000000        0.000000   
14  0.000

In [124]:
# set number of clusters
clusters = 5

toronto_clust = toronto_freq.drop('Neighbourhood', axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters = clusters, random_state=0).fit(toronto_clust)


kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [125]:
commondf['clusters'] = kmeans.labels_

In [126]:
commondf

Neighbourhood        CommonVenue1  \
0                                         Berczy Park         Coffee Shop   
1        Brockton, Parkdale Village, Exhibition Place                Café   
2   Business reply mail Processing Centre, South C...  Light Rail Station   
3   CN Tower, King and Spadina, Railway Lands, Har...      Airport Lounge   
4                                  Central Bay Street         Coffee Shop   
5                                            Christie       Grocery Store   
6                                Church and Wellesley         Coffee Shop   
7                      Commerce Court, Victoria Hotel         Coffee Shop   
8                                          Davisville        Dessert Shop   
9                                    Davisville North      Breakfast Spot   
10                       Dufferin, Dovercourt Village              Bakery   
11             First Canadian Place, Underground city         Coffee Shop   
12    Forest Hill North & West, Forest Hill Road Park            Bus Line   
13                           Garden District, Ryerson      Clothing Store   
14  Harbourfront East, Union Station, Toronto Islands         Coffee Shop   
15                      High Park, The Junction South                Café   
16                     India Bazaar, The Beaches West                Park   
17          Kensington Market, Chinatown, Grange Park                Café   
18                                      Lawrence Park                Park   
19                           Little Portugal, Trinity                 Bar   
20                        Moore Park, Summerhill East               Trail   
21                  North Toronto West, Lawrence Park      Clothing Store   
22                             Parkdale, Roncesvalles      Breakfast Spot   
23        Queen's Park, Ontario Provincial Government         Coffee Shop   
24                          Regent Park, Harbourfront         Coffee Shop   
25                           Richmond, Adelaide, King         Coffee Shop   
26                                           Rosedale                Park   
27                                           Roselawn              Garden   
28                                 Runnymede, Swansea                Café   
29                                     St. James Town                Café   
30                        St. James Town, Cabbagetown              Bakery   
31                                     Stn A PO Boxes         Coffee Shop   
32                                    Studio District                Café   
33  Summerhill West, Rathnelly, South Hill, Forest...         Coffee Shop   
34                The Annex, North Midtown, Yorkville                Café   
35                                        The Beaches         Coffee Shop   
36                       The Danforth West, Riverdale    Greek Restaurant   
37           Toronto Dominion Centre, Design Exchange         Coffee Shop   
38                     University of Toronto, Harbord                Café   

           CommonVenue2                   CommonVenue3  \
0                Bakery                       Beer Bar   
1        Breakfast Spot                    Coffee Shop   
2         Auto Workshop                        Brewery   
3       Airport Service               Airport Terminal   
4    Italian Restaurant                           Café   
5                  Café                           Park   
6   Japanese Restaurant               Sushi Restaurant   
7                  Café                     Restaurant   
8           Pizza Place                 Sandwich Place   
9      Department Store              Food & Drink Shop   
10             Pharmacy                           Bank   
11                 Café                          Hotel   
12        Jewelry Store                           Park   
13          Coffee Shop                Bubble Tea Shop   
14             Aquarium                           Café   
15   Mexican Restaurant        

In [127]:
toronto_final = tor_data

toronto_final = toronto_final.join(commondf.set_index('Neighbourhood'), on='Neighbourhood')
toronto_final.head()

Postal Code           Borough                                Neighbourhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude    CommonVenue1       CommonVenue2  \
0  43.654260 -79.360636     Coffee Shop             Bakery   
1  43.662301 -79.389494     Coffee Shop              Diner   
2  43.657162 -79.378937  Clothing Store        Coffee Shop   
3  43.651494 -79.375418            Café        Coffee Shop   
4  43.676357 -79.293031     Coffee Shop  Health Food Store   

          CommonVenue3         CommonVenue4    CommonVenue5  ...  \
0                 Park                  Pub  Breakfast Spot  ...   
1  Arts & Crafts Store                 Bank             Bar  ...   
2      Bubble Tea Shop                 Café  Cosmetics Shop  ...   
3       Clothing Store  American Restaurant    Cocktail Bar  ...   
4         Neighborhood                  Pub           Trail  ...   

         CommonVenue7         CommonVenue8        CommonVenue9  \
0             Theater         Antique Shop         Art Gallery   
1       Burrito Place                 Café  College Auditorium   
2  Italian Restaurant  Japanese Restaurant           Bookstore   
3          Restaurant             Beer Bar      Breakfast Spot   
4             Airport   Airport Food Court        Airport Gate   

       CommonVenue10         CommonVenue11        CommonVenue12  \
0   Asian Restaurant                  Bank           Beer Store   
1  College Cafeteria              Creperie  Distribution Center   
2  Electronics Store  Fast Food Restaurant       Lingerie Store   
3           Creperie      Department Store       Farmers Market   
4     Airport Lounge       Airport Service     Airport Terminal   

               CommonVenue13          CommonVenue14        CommonVenue15  \
0             Chocolate Shop           Dessert Shop  Distribution Center   
1        Fried Chicken Joint  General Entertainment                  Gym   
2  Middle Eastern Restaurant            Pizza Place                Plaza   
3                  Gastropub                    Gym                Hotel   
4        American Restaurant           Antique Shop             Aquarium   

  clusters  
0        0  
1        0  
2        0  
3        0  
4        0  

[5 rows x 21 columns]

In [128]:
toronto_final.shape

(39, 21)

In [129]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [130]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Borough'], toronto_final['clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the clusters

In [131]:
toronto_final

Postal Code           Borough  \
0          M5A  Downtown Toronto   
1          M7A  Downtown Toronto   
2          M5B  Downtown Toronto   
3          M5C  Downtown Toronto   
4          M4E      East Toronto   
5          M5E  Downtown Toronto   
6          M5G  Downtown Toronto   
7          M6G  Downtown Toronto   
8          M5H  Downtown Toronto   
9          M6H      West Toronto   
10         M5J  Downtown Toronto   
11         M6J      West Toronto   
12         M4K      East Toronto   
13         M5K  Downtown Toronto   
14         M6K      West Toronto   
15         M4L      East Toronto   
16         M5L  Downtown Toronto   
17         M4M      East Toronto   
18         M4N   Central Toronto   
19         M5N   Central Toronto   
20         M4P   Central Toronto   
21         M5P   Central Toronto   
22         M6P      West Toronto   
23         M4R   Central Toronto   
24         M5R   Central Toronto   
25         M6R      West Toronto   
26         M4S   Central Toronto   
27         M5S  Downtown Toronto   
28         M6S      West Toronto   
29         M4T   Central Toronto   
30         M5T  Downtown Toronto   
31         M4V   Central Toronto   
32         M5V  Downtown Toronto   
33         M4W  Downtown Toronto   
34         M5W  Downtown Toronto   
35         M4X  Downtown Toronto   
36         M5X  Downtown Toronto   
37         M4Y  Downtown Toronto   
38         M7Y      East Toronto   

                                        Neighbourhood   Latitude  Longitude  \
0                           Regent Park, Harbourfront  43.654260 -79.360636   
1         Queen's Park, Ontario Provincial Government  43.662301 -79.389494   
2                            Garden District, Ryerson  43.657162 -79.378937   
3                                      St. James Town  43.651494 -79.375418   
4                                         The Beaches  43.676357 -79.293031   
5                                         Berczy Park  43.644771 -79.373306   
6                                  Central Bay Street  43.657952 -79.387383   
7                                            Christie  43.669542 -79.422564   
8                            Richmond, Adelaide, King  43.650571 -79.384568   
9                        Dufferin, Dovercourt Village  43.669005 -79.442259   
10  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752   
11                           Little Portugal, Trinity  43.647927 -79.419750   
12                       The Danforth West, Riverdale  43.679557 -79.352188   
13           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576   
14       Brockton, Parkdale Village, Exhibition Place  43.636847 -79.428191   
15                     India Bazaar, The Beaches West  43.668999 -79.315572   
16                     Commerce Court, Victoria Hotel  43.648198 -79.379817   
17                                    Studio District  43.659526 -79.340923   
18                                      Lawrence Park  43.728020 -79.388790   
19                                           Roselawn  43.711695 -79.416936   
20                                   Davisville North  43.712751 -79.390197   
21    Forest Hill North & West, Forest Hill Road Park  43.696948 -79.411307   
22                      High Park, The Junction South  43.661608 -79.464763   
23                  North Toronto West, Lawrence Park  43.715383 -79.405678   
24                The Annex, North Midtown, Yorkville  43.672710 -79.405678   
25                             Parkdale, Roncesvalles  43.648960 -79.456325   
26                                         Davisville  43.704324 -79.388790   
27                     University of Toronto, Harbord  43.662696 -79.400049   
28                                 Runnymede, Swansea  43.651571 -79.484450   
29                        Moore Park, Summerhill East  43.689574 -79.383160   
30          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049   
31  Summerhill West, Rathnelly, So

In [132]:
# Cluster 1
toronto_final.loc[toronto_final['clusters'] == 0, toronto_final.columns[[1] + [2] + list(range(5, toronto_final.shape[1] - 1))]]

Borough                                      Neighbourhood  \
0   Downtown Toronto                          Regent Park, Harbourfront   
1   Downtown Toronto        Queen's Park, Ontario Provincial Government   
2   Downtown Toronto                           Garden District, Ryerson   
3   Downtown Toronto                                     St. James Town   
4       East Toronto                                        The Beaches   
5   Downtown Toronto                                        Berczy Park   
6   Downtown Toronto                                 Central Bay Street   
7   Downtown Toronto                                           Christie   
8   Downtown Toronto                           Richmond, Adelaide, King   
9       West Toronto                       Dufferin, Dovercourt Village   
10  Downtown Toronto  Harbourfront East, Union Station, Toronto Islands   
11      West Toronto                           Little Portugal, Trinity   
12      East Toronto                       The Danforth West, Riverdale   
13  Downtown Toronto           Toronto Dominion Centre, Design Exchange   
14      West Toronto       Brockton, Parkdale Village, Exhibition Place   
15      East Toronto                     India Bazaar, The Beaches West   
16  Downtown Toronto                     Commerce Court, Victoria Hotel   
17      East Toronto                                    Studio District   
20   Central Toronto                                   Davisville North   
22      West Toronto                      High Park, The Junction South   
23   Central Toronto                  North Toronto West, Lawrence Park   
24   Central Toronto                The Annex, North Midtown, Yorkville   
25      West Toronto                             Parkdale, Roncesvalles   
26   Central Toronto                                         Davisville   
27  Downtown Toronto                     University of Toronto, Harbord   
28      West Toronto                                 Runnymede, Swansea   
30  Downtown Toronto          Kensington Market, Chinatown, Grange Park   
31   Central Toronto  Summerhill West, Rathnelly, South Hill, Forest...   
32  Downtown Toronto  CN Tower, King and Spadina, Railway Lands, Har...   
34  Downtown Toronto                                     Stn A PO Boxes   
35  Downtown Toronto                        St. James Town, Cabbagetown   
36  Downtown Toronto             First Canadian Place, Underground city   
37  Downtown Toronto                               Church and Wellesley   
38      East Toronto  Business reply mail Processing Centre, South C...   

          CommonVenue1         CommonVenue2                   CommonVenue3  \
0          Coffee Shop               Bakery                           Park   
1          Coffee Shop                Diner            Arts & Crafts Store   
2       Clothing Store          Coffee Shop                Bubble Tea Shop   
3                 Café          Coffee Shop                 Clothing Store   
4          Coffee Shop    Health Food Store                   Neighborhood   
5          Coffee Shop               Bakery                       Beer Bar   
6          Coffee Shop   Italian Restaurant                           Café   
7        Grocery Store                 Café                           Park   
8          Coffee Shop                 Café                 Clothing Store   
9               Bakery             Pharmacy                           Bank   
10         Coffee Shop             Aquarium                           Café   
11                 Bar     Asian Restaurant                           Café   
12    Greek Restaurant          Coffee Shop             Italian Restaurant   
13         Coffee Shop                 Café                          Hotel   
14                Café       Breakfast Spot                    Coffee Shop   
15                Park           Board Shop                        Brewery   
16         Coffee Shop                 Café                  

In [133]:
# Cluster 2
toronto_final.loc[toronto_final['clusters'] == 1, toronto_final.columns[[1] + [2] + list(range(5, toronto_final.shape[1] - 1))]]

Borough                Neighbourhood CommonVenue1  \
29  Central Toronto  Moore Park, Summerhill East        Trail   

         CommonVenue2 CommonVenue3        CommonVenue4  CommonVenue5  \
29  Afghan Restaurant      Airport  Airport Food Court  Airport Gate   

      CommonVenue6     CommonVenue7      CommonVenue8         CommonVenue9  \
29  Airport Lounge  Airport Service  Airport Terminal  American Restaurant   

   CommonVenue10 CommonVenue11 CommonVenue12 CommonVenue13  \
29  Antique Shop      Aquarium   Art Gallery    Art Museum   

          CommonVenue14     CommonVenue15  
29  Arts & Crafts Store  Asian Restaurant

In [134]:
# Cluster 3
toronto_final.loc[toronto_final['clusters'] == 2, toronto_final.columns[[1] + [2] + list(range(5, toronto_final.shape[1] - 1))]]

Borough Neighbourhood CommonVenue1  CommonVenue2  \
19  Central Toronto      Roselawn       Garden  Home Service   

         CommonVenue3 CommonVenue4        CommonVenue5  CommonVenue6  \
19  Afghan Restaurant      Airport  Airport Food Court  Airport Gate   

      CommonVenue7     CommonVenue8      CommonVenue9        CommonVenue10  \
19  Airport Lounge  Airport Service  Airport Terminal  American Restaurant   

   CommonVenue11 CommonVenue12 CommonVenue13 CommonVenue14  \
19  Antique Shop      Aquarium   Art Gallery    Art Museum   

          CommonVenue15  
19  Arts & Crafts Store

In [135]:
# Cluster 4
toronto_final.loc[toronto_final['clusters'] == 3, toronto_final.columns[[1] + [2] +  list(range(5, toronto_final.shape[1] - 1))]]

Borough                                    Neighbourhood  \
21  Central Toronto  Forest Hill North & West, Forest Hill Road Park   

   CommonVenue1   CommonVenue2 CommonVenue3      CommonVenue4 CommonVenue5  \
21     Bus Line  Jewelry Store         Park  Sushi Restaurant        Trail   

         CommonVenue6 CommonVenue7        CommonVenue8  CommonVenue9  \
21  Afghan Restaurant      Airport  Airport Food Court  Airport Gate   

     CommonVenue10    CommonVenue11     CommonVenue12        CommonVenue13  \
21  Airport Lounge  Airport Service  Airport Terminal  American Restaurant   

   CommonVenue14 CommonVenue15  
21  Antique Shop      Aquarium

In [136]:
# Cluster 5
toronto_final.loc[toronto_final['clusters'] == 4, toronto_final.columns[[1] + [2] +  list(range(5, toronto_final.shape[1] - 1))]]

Borough  Neighbourhood CommonVenue1 CommonVenue2 CommonVenue3  \
18   Central Toronto  Lawrence Park         Park     Bus Line  Swim School   
33  Downtown Toronto       Rosedale         Park   Playground        Trail   

         CommonVenue4 CommonVenue5        CommonVenue6  CommonVenue7  \
18  Afghan Restaurant      Airport  Airport Food Court  Airport Gate   
33  Afghan Restaurant      Airport  Airport Food Court  Airport Gate   

      CommonVenue8     CommonVenue9     CommonVenue10        CommonVenue11  \
18  Airport Lounge  Airport Service  Airport Terminal  American Restaurant   
33  Airport Lounge  Airport Service  Airport Terminal  American Restaurant   

   CommonVenue12 CommonVenue13 CommonVenue14 CommonVenue15  
18  Antique Shop      Aquarium   Art Gallery    Art Museum  
33  Antique Shop      Aquarium   Art Gallery    Art Museum

As it is shown in cluster 1 most of bakeries, restaurants and coffe shops are clustered together. While the rest of the clusters are less populated and since most of them include venues as airport they could be clustered together into one